<a href="https://colab.research.google.com/github/peaugust/INE5644-data-mining/blob/main/Data_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise do dataset: Brazilian E-Commerce Public Dataset by Olist

Importando as dependencias a serem utilizadas

In [4]:
!git clone https://github.com/peaugust/INE5644-data-mining.git
import seaborn as sns
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('max_rows', 200)
pd.set_option('display.max_colwidth', 150)


Cloning into 'INE5644-data-mining'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 48 (delta 11), reused 18 (delta 6), pack-reused 22
Unpacking objects: 100% (48/48), done.
Checking out files: 100% (24/24), done.


### Iremos analisar todas as tabelas buscando remover itens que sejam nulos, itens repetidos que possam prejudicar a analise, como tambem eliminaremos colunas que nao sao uteis para a nossa analise.

In [5]:
customers = pd.read_csv('./data-2/olist_customers_dataset.csv', dtype={'customer_zip_code_prefix': str})
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


In [6]:
# Removendo customer_unique_id do conjunto de dados
customers = customers.drop(columns=['customer_unique_id'])    
duplicated_customers = customers[customers.duplicated(subset=['customer_id'])==True]
print(f'Temos {duplicated_customers.size} duplicatas nesse dataset')
customers

Temos 0 duplicatas nesse dataset


,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP
...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,03937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,06764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,92120,canoas,RS


In [7]:
# Limpando as geolocalizações repetidas e usando uma coordenada média
geolocation = pd.read_csv('./data-2/olist_geolocation_dataset.csv', dtype={'geolocation_zip_code_prefix': str})
geolocation = geolocation.groupby(['geolocation_zip_code_prefix', 'geolocation_state']).agg(lat=('geolocation_lat', 'median'),lon=('geolocation_lng', 'median')).reset_index()
geolocation['lat_lon'] = tuple(zip(geolocation.lat, geolocation.lon))
geolocation.drop(columns=['lat', 'lon'], inplace=True)

In [8]:
duplicated_geolocation = geolocation[geolocation.duplicated(subset=['geolocation_zip_code_prefix', 'geolocation_state'])==True]
print(f'Temos {duplicated_geolocation.size} duplicatas nesse dataset')
geolocation

Temos 0 duplicatas nesse dataset


,geolocation_zip_code_prefix,geolocation_state,lat_lon
0,01001,SP,"(-23.550380539269455, -46.634026997778314)"
1,01002,SP,"(-23.548551065785627, -46.635072390789965)"
2,01003,SP,"(-23.548976528997674, -46.635313112268456)"
3,01004,SP,"(-23.549534948092827, -46.634770582289825)"
4,01005,SP,"(-23.54961248124544, -46.636532025535224)"
...,...,...,...
19018,99960,RS,"(-27.95379724325779, -52.02964105736787)"
19019,99965,RS,"(-28.179541888501852, -52.03555069608984)"
19020,99970,RS,"(-28.343272557292238, -51.87373383061928)"
19021,99980,RS,"(-28.38809203555391, -51.846880216369655)"


In [13]:
order_items = pd.read_csv('./data-2/olist_order_items_dataset.csv')
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [17]:
duplicated_order_items = order_items[order_items.duplicated(subset=['order_id', 'order_item_id'])==True]
print(f'Temos {duplicated_order_items.size} duplicatas nesse dataset')
order_items.groupby('order_item_id').count()

Temos 0 duplicatas nesse dataset


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value
order_item_id,,,,,,
1,98666,98666,98666,98666,98666,98666
2,9803,9803,9803,9803,9803,9803
3,2287,2287,2287,2287,2287,2287
4,965,965,965,965,965,965
5,460,460,460,460,460,460
6,256,256,256,256,256,256
7,58,58,58,58,58,58
8,36,36,36,36,36,36
9,28,28,28,28,28,28


In [19]:
order_reviews = pd.read_csv('./data-2/olist_order_reviews_dataset.csv', parse_dates=['review_creation_date', 
                                                 'review_answer_timestamp'])
order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     11568 non-null  object        
 4   review_comment_message   40977 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB


In [20]:
duplicated_order_reviews = order_reviews[order_reviews.duplicated(subset=['order_id'])==True]
print(f'Temos {len(duplicated_order_reviews)} duplicatas nesse dataset')
order_reviews_cleaned = order_reviews.drop_duplicates(subset=['order_id'])
print(f'Clean: {len(order_reviews_cleaned)}, Dirty: {len(order_reviews)}, Diff {len(order_reviews) - len(order_reviews_cleaned)}')

duplicated_orders_reviews_after_clean = order_reviews_cleaned[order_reviews_cleaned.duplicated(subset=['order_id']) == True]
print(f'Temos {len(duplicated_orders_reviews_after_clean)} duplicatas nesse dataset')
order_reviews_cleaned = order_reviews_cleaned.drop(columns=['review_comment_title', 'review_comment_message'])
order_reviews_cleaned.info()

Temos 551 duplicatas nesse dataset
Clean: 98673, Dirty: 99224, Diff 551
Temos 0 duplicatas nesse dataset
<class 'pandas.core.frame.DataFrame'>
Int64Index: 98673 entries, 0 to 99223
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98673 non-null  object        
 1   order_id                 98673 non-null  object        
 2   review_score             98673 non-null  int64         
 3   review_creation_date     98673 non-null  datetime64[ns]
 4   review_answer_timestamp  98673 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 4.5+ MB


In [21]:
orders = pd.read_csv('./data-2/olist_orders_dataset.csv')
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [ ]:
duplicated_orders = orders[orders.duplicated(subset=['order_id'])==True]
print(f'Temos {len(duplicated_orders)} duplicatas nesse dataset')
orders

Temos 0 duplicatas nesse dataset


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [29]:
products = pd.read_csv('./data-2/olist_products_dataset.csv')
products.info()
products = products.drop(columns=['product_name_lenght', 'product_description_lenght', 'product_photos_qty'])
products.info()
products

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,700.0,31.0,13.0,20.0


In [30]:
duplicated_products = products[products.duplicated(subset=['product_id'])==True]
print(f'Temos {len(duplicated_products)} duplicatas nesse dataset')
products

Temos 0 duplicatas nesse dataset


,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,700.0,31.0,13.0,20.0


In [31]:
sellers = pd.read_csv('./data-2/olist_sellers_dataset.csv')
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [32]:
duplicated_sellers = sellers[sellers.duplicated(subset=['seller_id'])==True]
print(f'Temos {len(duplicated_sellers)} duplicatas nesse dataset')
sellers

Temos 0 duplicatas nesse dataset


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [33]:
product_category_name_translation = pd.read_csv('./data-2/product_category_name_translation.csv')
product_category_name_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [ ]:
product_category_name_translation

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
5,esporte_lazer,sports_leisure
6,perfumaria,perfumery
7,utilidades_domesticas,housewares
8,telefonia,telephony
9,relogios_presentes,watches_gifts
